<a href="https://colab.research.google.com/github/PernasA/TP2_SOA/blob/master/HPC/Cuaderno_2_Martes_grupo6a_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### 1. Introducción

El Ejercicio 2 del TP3 tiene como objetivo probar la diferencia de velocidad de procesamiento que hay entre procesamiento secuencial y binario con paralelo. Para ello, se generará un vector con valores enteros que progresan aleatoriamente entre 1 y 3 valores. Esto quiere decir que, a medida que avanzan las posiciones del vector, cada vez los valores van a ser más grandes. Se pedirá al usuario que defina el tamaño del mismo. También se le solicitará que ingrese el número que se quiere buscar en el vector.
Con estos datos, el programa buscará al elemento y si lo encuentra dirá cuál fue la posición donde apareció. El número a buscar puede aparecer o una o ninguna vez (depende de la aleatoriedad de la creación del vector).

### 2. Armado del ambiente


In [1]:
#Instalacion de la biblioteca 'PYCUDA'
!pip install pycuda

     |████████████████████████████████| 1.7 MB 11.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 66 kB 4.7 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627577 sha256=30dc941c62aa742f9de296960fe8493ec4c805d7b7a75c8223c8cce2165ced12
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=27c78efa1e4c992068f5df26c7c786b53c7df1098b7fc413b8587c1fed2f8220
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


In [8]:
 try:
  import numpy
  #Genero un formulario para que el usuario pueda seleccionar las variables
  #@markdown ### Seleccione el tamaño del vector:
  tam_vector = 10 #@param {type:"slider", min:1, max:200000, step:10}
  #@markdown ---
  #@markdown ### Seleccione el número a buscar:
  nro_a_buscar = 10 #@param {type:"slider", min:0, max:100000, step:1}

  #Cargo en un vector del tamaño ingresado elementos que aumentan
  #aleatoriamente entre 1 y 3 valores.
  vector_cpu= numpy.empty([tam_vector], dtype=int)
  acumulador = 0
  for number in range(tam_vector):
    valorRandom = numpy.random.randint(1,4, size = 1)
    acumulador += valorRandom
    vector_cpu[number] =  acumulador

  #Muestro el vector para una mayor comprension del ejercicio  
  print("El vector es: ",vector_cpu.tolist())
except Exception as e:
  print("Error", e )
  sys.exit()

El vector es:  [2, 5, 7, 10, 11, 13, 16, 17, 18, 19]


### 3. Desarrollo
# CPU - Búsqueda Secuencial


In [30]:
#--------------------------------------BIBLIOTECAS--------------------------------------#
import matplotlib.pyplot as plt
from datetime import datetime

import sys
try:
  #@title Ingrese la url de una imagen:
  #Obtencion del tiempo inicial
  tiempo_total = datetime.now()
  
  #Declaracion de funcion que realiza el pasaje del tiempo obtenido mediante datetime.now(), a milisegundos.
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  #---------------------------------------CODIGO--------------------------------------------#  

  tiempo_cpu = datetime.now()

  #Aplico el algoritmo de búsqueda secuencial
  aparicion = -1
  contador = 0
  while contador < len(vector_cpu):
    if(vector_cpu[contador] == nro_a_buscar):
      aparicion = contador+1
      contador = len(vector_cpu)+1
    elif vector_cpu[contador]> nro_a_buscar:
      contador = len(vector_cpu)+1
    contador += 1

  #Muestro los resultados obtenidos
  print( "Cantidad de elementos:", len(vector_cpu) )
  print("Tiempo CPU - Búsqueda Secuencial:", tiempo_en_ms( datetime.now() - tiempo_cpu   ), "[ms]" )

  if aparicion != -1:
    print("Se encontró por CPU el valor:",nro_a_buscar)
    print("La posición donde aparece es:", aparicion)
  else:
    print("No se encontró por CPU el valor:",nro_a_buscar)

  #Calculo del tiempo total de ejecucion del programa
  tiempo_total = datetime.now() - tiempo_total

  print("Tiempo total de ejecucion: ", tiempo_en_ms( tiempo_total ), "[ms]" )

except Exception as e:
  print("Error", e )
  sys.exit()

Cantidad de elementos: 10
Tiempo CPU - Búsqueda Secuencial: 0.259 [ms]
Se encontró por CPU el valor: 10
La posición donde aparece es: 4
Tiempo total de ejecucion:  0.528 [ms]


# CPU - Búsqueda Binaria

In [19]:
#--------------------------------------BIBLIOTECAS--------------------------------------#
import matplotlib.pyplot as plt
from datetime import datetime
import numpy
import sys
try:
  #Obtencion del tiempo inicial
  tiempo_total = datetime.now()
  
  #Declaracion de funcion que realiza el pasaje del tiempo obtenido mediante datetime.now(), a milisegundos.
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  #---------------------------------------CODIGO--------------------------------------------#
  # Definimos el vector y lo cargamos con valores aleatorios del 0 al 100

  tiempo_cpu = datetime.now()

   #Aplico el algoritmo de búsqueda binaria con comentarios en ingles obtenidos de la bibliografía.
  aparicion = -1
  contador = 1

  # Start with the entire sequence of elements.
  low = 0
  high = len(vector_cpu) - 1
  # Repeatedly subdivide the sequence in half until the nro_a_buscar is found.
  while low <= high :
    # Find the midpoint of the sequence.
    mid = (high + low) // 2
    # Does the midpoint contain the nro_a_buscar?
    if vector_cpu[mid] == nro_a_buscar :
      aparicion = mid+1
      low = high + 1
    # Or does the nro_a_buscar precede the midpoint?
    elif nro_a_buscar < vector_cpu[mid] :
      high = mid - 1
    # Or does it nro_a_buscar the midpoint?
    else :
      low = mid + 1

  #Muestro los resultados y tiempos obtenidos
  print( "Cantidad de elementos:", len(vector_cpu) )
  print("Tiempo CPU - Búsqueda Binaria:", tiempo_en_ms( datetime.now() - tiempo_cpu   ), "[ms]" )

  if aparicion != -1:
    print("Se encontró por CPU el valor:",nro_a_buscar)
    print("La posición donde aparece es:", aparicion)
  else:
    print("No se encontró por CPU el valor:",nro_a_buscar)

  #Calculo del tiempo total de ejecucion del programa
  tiempo_total = datetime.now() - tiempo_total

  print("Tiempo total de ejecucion: ", tiempo_en_ms( tiempo_total ), "[ms]" )

except Exception as e:
  print("Error", e )
  sys.exit()

Cantidad de elementos: 10
Tiempo CPU - Búsqueda Binaria: 0.186 [ms]
Se encontró por CPU el valor: 10
La posición donde aparece es: 4
Tiempo total de ejecucion:  1.17 [ms]


# GPU - Búsqueda Paralelizada

In [20]:
try:
  #Importo las bibliotecas necesarias para la ejecución del ejercicio
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()

  import matplotlib.pyplot as plt
  import sys
  import numpy
  import pycuda.driver as cuda
  import pycuda.autoinit
  from pycuda.compiler import SourceModule

  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  # Defino un vector de un sólo elemento en 0.
  valor_aux = numpy.zeros((1,), dtype=int);
  valor_aux = valor_aux.astype(numpy.int32())

  vector_cpu = vector_cpu.astype(numpy.int32())
  vec_pos_cpu= numpy.empty_like(valor_aux)

  #Reservo memoria en los vectores de gpu
  vec_aparicion_gpu = cuda.mem_alloc(vec_pos_cpu.nbytes)
  vector_gpu        = cuda.mem_alloc( vector_cpu.nbytes )

  #Copio la memoria al gpu
  cuda.memcpy_htod( vec_aparicion_gpu, valor_aux)
  cuda.memcpy_htod( vector_gpu, vector_cpu )

  # CPU - Defino la función kernel que ejecutará en GPU.
  module = SourceModule("""
  __global__ void kernel( int tam_vector, int *vector, int nro_a_buscar, int *aparicion )
  {
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    if( idx < tam_vector )
    {
      if (vector[idx] == nro_a_buscar) {
        aparicion[0]=idx +1;
      }
    }
  }
  """)
  # CPU - Genero la función kernel.
  kernel = module.get_function("kernel")

  #Obtengo tiempo anterior a ejecucion en gpu
  tiempo_GPU_img = datetime.now()

  #Calculo de hilos y bloques
  dim_hilo = 1000
  dim_bloque = numpy.int( (len(vector_cpu)+dim_hilo-1) / dim_hilo )

  # GPU - Ejecuta el kernel.
  kernel( numpy.int32(len(vector_cpu)), vector_gpu, 
          numpy.int32(nro_a_buscar), 
          vec_aparicion_gpu,
          block=( dim_hilo, 1, 1 ),
          grid= (dim_bloque, 1, 1) )

  #Paso la informacion a cpu
  cuda.memcpy_dtoh( vec_pos_cpu, vec_aparicion_gpu )
  result= vec_pos_cpu.tolist()

  #Muestro los tiempos y resultados obtenidos
  tiempo_GPU_img = datetime.now() - tiempo_GPU_img
  aparicion = result[0]
  print( "Cantidad de elementos:", len(vector_cpu) )
  print( "Thread x:", dim_hilo,", Bloque x:", dim_bloque )
  print("Tiempo GPU - Búsqueda Paralelizada  : ", tiempo_en_ms( tiempo_GPU_img ), "[ms]" )
  
  if aparicion != 0:
    print("Se encontró por GPU el valor:",nro_a_buscar)
    print("La posición donde aparece es:",aparicion)
  else:
    print("No se encontró por GPU el valor:",nro_a_buscar)

  tiempo_total = datetime.now() - tiempo_total

  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )

except Exception as e:
  print("Error: ", e)
  sys.exit()

Cantidad de elementos: 10
Thread x: 1000 , Bloque x: 1
Tiempo GPU - Búsqueda Paralelizada  :  0.115 [ms]
Se encontró por GPU el valor: 10
La posición donde aparece es: 4
Tiempo TOTAL:  2.649 [ms]


# 4. Métricas
Hay que aclarar que los métodos de búsqueda secuencial y búsqueda binaria son sensibles a la entrada. En el secuencial, mientras mayor es la nro_a_buscar, ,mucho mayor serán los tiempos de búsqueda. Tanto en búsqueda binaria como en secuencial, si el elemento no se encuentra en el vector sus tiempos serán levemente mayores.

Cant Elementos|Nro a buscar|Tiempo CPU - Secuencial | Tiempo CPU - Binaria | Tiempo GPU - Paralelizada
--------|------|------------|------------|-------
10      |10    |0.26 [ms]  |  0.19 [ms]| 0.115 [ms]  
1.000   |100   | 1.0[ms]    |  0.3   [ms]| 0.12 [ms]
10.000  |1000  | 1.3   [ms] |  0.5   [ms]| 0.16 [ms]
100.000 |100000|240.0 [ms]|    0.9   [ms]| 0.18 [ms]
200.000 |50000 |29 [ms] |     1.1  [ms] | 0.18 [ms]
200.000 (sin aparicion) |100000|247 [ms] |   1.2    [ms] | 0.2 [ms]



# 5. Conclusiones

Observando las metricas, podemos concluir que los distintos tipos de búsqueda de los elementos en el vector influyen bastante en los tiempos.
En primer lugar, tenemos al método de búsqueda secuencial en CPU, que a medida que el número a buscar aumenta, sus tiempos aumentan considerablemente. Luego, con la forma de búsqueda binaria, los tiempos aumentan de forma constante pero mucho menos que los de la búsqueda secuencial.
Finalmente, la búsqueda paralelizada utilizando hilos gracias a CUDA, es la mejor de las tres. Se puede ver cómo se mantienen bastante estables los tiempos a pesar de aumentar la cantidad de elementos o el número a buscar. Se puede concluir que utilizar hilos para una búsqueda en un array es la mejor posibilidad, especialmente cuando el tamaño del array es más grande.

# 6. Bibliografía

[1] Manejo de vectores -Numpy: https://numpy.org/doc/stable/reference/generated/numpy.zeros.html

[2] Cuda - Documentación Oficial Nvidia:  https://docs.nvidia.com/cuda/

[3] Python basico UNLaM: https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb

[4] 	D. Munch, «Parallel programming with CUDA. Architecture, Analysis, Application,» Universitat Karlsruhe (TH), Alemania, 2008.

[5] 	R. D. Necaise, Data Structures and Algorithms Using Python, EE UU: Jhon Wiley & Sons, Inc, 2011. 
